<a href="https://colab.research.google.com/github/donw385/DS-Unit-2-Sprint-3-Advanced-Regression/blob/master/DS_Unit_2_Sprint_Challenge_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Science Unit 2 Sprint Challenge 3

## Logistic Regression and Beyond

In this sprint challenge you will fit a logistic regression modeling the probability of an adult having an income above 50K. The dataset is available at UCI:

https://archive.ics.uci.edu/ml/datasets/adult

Your goal is to:

1. Load, validate, and clean/prepare the data.
2. Fit a logistic regression model
3. Answer questions based on the results (as well as a few extra questions about the other modules)

Don't let the perfect be the enemy of the good! Manage your time, and make sure to get to all parts. If you get stuck wrestling with the data, simplify it (if necessary, drop features or rows) so you're able to move on. If you have time at the end, you can go back and try to fix/improve.

### Hints

It has a variety of features - some are continuous, but many are categorical. You may find [pandas.get_dummies](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html) (a method to one-hot encode) helpful!

The features have dramatically different ranges. You may find [sklearn.preprocessing.minmax_scale](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.minmax_scale.html#sklearn.preprocessing.minmax_scale) helpful!

## Part 1 - Load, validate, and prepare data

The data is available at: https://archive.ics.uci.edu/ml/datasets/adult

Load it, name the columns, and make sure that you've loaded the data successfully. Note that missing values for categorical variables can essentially be considered another category ("unknown"), and may not need to be dropped.

You should also prepare the data for logistic regression - one-hot encode categorical features as appropriate.

In [0]:
# all imports at beginning

import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.preprocessing import scale


pd.set_option('display.max_columns', 500)

In [47]:
#only using data file


df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data',header=None)
df.columns = ['age','workclass','fnlwgt','education','education_num','marital','occupation','relationship','race','sex','capitalgain','capitalloss','hours_week','home_country','income']


df.head(500)

,age,workclass,fnlwgt,education,education_num,marital,occupation,relationship,race,sex,capitalgain,capitalloss,hours_week,home_country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
5,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K
6,49,Private,160187,9th,5,Married-spouse-absent,Other-service,Not-in-family,Black,Female,0,0,16,Jamaica,<=50K
7,52,Self-emp-not-inc,209642,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,45,United-States,>50K
8,31,Private,45781,Masters,14,Never-married,Prof-specialty,Not-in-family,White,Female,14084,0,50,United-States,>50K
9,42,Private,159449,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,5178,0,40,United-States,>50K


In [5]:
df.shape

(32561, 15)

In [39]:
df['income_code'].value_counts()

0    24720
1     7841
Name: income_code, dtype: int64

In [40]:
df['income'].value_counts()

 <=50K    24720
 >50K      7841
Name: income, dtype: int64

In [0]:
#encode categorical labels

df['workclass'] = df['workclass'].astype('category')
df['work_code'] = df['workclass'].cat.codes

df['marital'] = df['marital'].astype('category')
df['marital_code'] = df['marital'].cat.codes

df['occupation'] = df['occupation'].astype('category')
df['occupation_code'] = df['occupation'].cat.codes

df['relationship'] = df['relationship'].astype('category')
df['relationship_code'] = df['relationship'].cat.codes

df['race'] = df['race'].astype('category')
df['race_code'] = df['race'].cat.codes

df['sex'] = df['sex'].astype('category')
df['sex_code'] = df['sex'].cat.codes

df['sex'] = df['sex'].astype('category')
df['sex_code'] = df['sex'].cat.codes

df['home_country'] = df['home_country'].astype('category')
df['home_country_code'] = df['home_country'].cat.codes


df['income'] = df['income'].astype('category')
df['income_code'] = df['income'].cat.codes

In [49]:
df.head()



,age,workclass,fnlwgt,education,education_num,marital,occupation,relationship,race,sex,capitalgain,capitalloss,hours_week,home_country,income,work_code,marital_code,occupation_code,relationship_code,race_code,sex_code,home_country_code,income_code
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K,7,4,1,1,4,1,39,0
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K,6,2,4,0,4,1,39,0
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K,4,0,6,1,4,1,39,0
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K,4,2,6,0,2,1,39,0
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K,4,2,10,5,2,0,5,0


In [41]:
df.dtypes

age                     int64
workclass            category
fnlwgt                  int64
education              object
education_num           int64
marital              category
occupation           category
relationship         category
race                 category
sex                  category
capitalgain             int64
capitalloss             int64
hours_week              int64
home_country         category
income               category
work_code                int8
marital_code             int8
occupation_code          int8
relationship_code        int8
race_code                int8
sex_code                 int8
home_country_code        int8
income_code              int8
dtype: object

In [42]:
#count is equal, value types seem good

df.describe()

,age,fnlwgt,education_num,capitalgain,capitalloss,hours_week,work_code,marital_code,occupation_code,relationship_code,race_code,sex_code,home_country_code,income_code
count,32561.000000,3.256100e+04,32561.000000,32561.000000,32561.000000,32561.000000,32561.000000,32561.000000,32561.000000,32561.000000,32561.000000,32561.000000,32561.000000,32561.000000
mean,38.581647,1.897784e+05,10.080679,1077.648844,87.303830,40.437456,3.868892,2.611836,6.572740,1.446362,3.665858,0.669205,36.718866,0.240810
std,13.640433,1.055500e+05,2.572720,7385.292085,402.960219,12.347429,1.455960,1.506222,4.228857,1.606771,0.848806,0.470506,7.823782,0.427581
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,28.000000,1.178270e+05,9.000000,0.000000,0.000000,40.000000,4.000000,2.000000,3.000000,0.000000,4.000000,0.000000,39.000000,0.000000
50%,37.000000,1.783560e+05,10.000000,0.000000,0.000000,40.000000,4.000000,2.000000,7.000000,1.000000,4.000000,1.000000,39.000000,0.000000
75%,48.000000,2.370510e+05,12.000000,0.000000,0.000000,45.000000,4.000000,4.000000,10.000000,3.000000,4.000000,1.000000,39.000000,0.000000
max,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000,8.000000,6.000000,14.000000,5.000000,4.000000,1.000000,41.000000,1.000000


In [43]:
df.columns

Index(['age', 'workclass', 'fnlwgt', 'education', 'education_num', 'marital',
       'occupation', 'relationship', 'race', 'sex', 'capitalgain',
       'capitalloss', 'hours_week', 'home_country', 'income', 'work_code',
       'marital_code', 'occupation_code', 'relationship_code', 'race_code',
       'sex_code', 'home_country_code', 'income_code'],
      dtype='object')

## Part 2 - Fit and present a Logistic Regression

Your data should now be in a state to fit a logistic regression. Use scikit-learn, define your `X` (independent variable) and `y`, and fit a model.

Then, present results - display coefficients in as interpretible a way as you can (hint - scaling the numeric features will help, as it will at least make coefficients more comparable to each other). If you find it helpful for interpretation, you can also generate predictions for cases (like our 5 year old rich kid on the Titanic) or make visualizations - but the goal is your exploration to be able to answer the question, not any particular plot (i.e. don't worry about polishing it).

It is *optional* to use `train_test_split` or validate your model more generally - that is not the core focus for this week. So, it is suggested you focus on fitting a model first, and if you have time at the end you can do further validation.

In [45]:
#logistic regression. score of 0.79 means 79% of y is explained by X variables

X = df[['age', 'fnlwgt',  'education_num',  'capitalgain',
       'capitalloss', 'hours_week', 'work_code',
       'marital_code', 'occupation_code', 'relationship_code', 'race_code',
       'sex_code', 'home_country_code']]
y = df.income_code

log_reg = LogisticRegression().fit(X, y)
log_reg.score(X, y)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.7922975338595252

In [55]:
#scale numeric

df_new = pd.DataFrame(scale(df[['age','fnlwgt','education_num','capitalgain','capitalloss', 'hours_week']]))

df_new.columns = ['age','fnlwgt','education_num','capitalgain','capitalloss', 'hours_week']
df_new.head()

df_new.shape

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DataConversionWarning: Data with input dtype int64 were all converted to float64 by the scale function.
  """Entry point for launching an IPython kernel.


(32561, 6)

In [0]:
#create new dataset

df_subset = df[[ 'work_code','marital_code', 'occupation_code', 'relationship_code', 'race_code',
       'sex_code', 'home_country_code','income_code']]

df_scaled = pd.concat([df_subset, df_new], axis=1)

df_scaled

In [62]:
#logistic regression using scaled numeric. score of 0.825 means 83% of y is explained by X variables

X = df_scaled[['age', 'fnlwgt',  'education_num',  'capitalgain',
       'capitalloss', 'hours_week', 'work_code',
       'marital_code', 'occupation_code', 'relationship_code', 'race_code',
       'sex_code', 'home_country_code']]
y = df_scaled.income_code

log_reg = LogisticRegression().fit(X, y)
log_reg.score(X, y)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.8250360861152913

In [68]:
#coefficients of logistic regression, organized into a table
coefficients = log_reg.coef_

Output = pd.DataFrame(coefficients)
Output.columns = ['age', 'fnlwgt',  'education_num',  'capitalgain',
       'capitalloss', 'hours_week', 'work_code',
       'marital_code', 'occupation_code', 'relationship_code', 'race_code',
       'sex_code', 'home_country_code']
Output

,age,fnlwgt,education_num,capitalgain,capitalloss,hours_week,work_code,marital_code,occupation_code,relationship_code,race_code,sex_code,home_country_code
0,0.463004,0.053061,0.855879,2.321904,0.273543,0.369323,-0.022047,-0.236281,0.010106,-0.120923,0.110382,0.887185,0.003366


In [56]:
df.shape

(32561, 23)

In [69]:
#linear regression for comparison. as you can see, the score of 0.26 is much lower than logistic regression score of 0.825

linear_reg = LinearRegression().fit(X, y)
linear_reg.score(X, y)

0.261077810304837

## Part 3 - Analysis, Interpretation, and Questions

### Based on your above model, answer the following questions

1. What are 3 features positively correlated with income above 50k?
2. What are 3 features negatively correlated with income above 50k?
3. Overall, how well does the model explain the data and what insights do you derive from it?

*These answers count* - that is, make sure to spend some time on them, connecting to your analysis above. There is no single right answer, but as long as you support your reasoning with evidence you are on the right track.

Note - scikit-learn logistic regression does *not* automatically perform a hypothesis test on coefficients. That is OK - if you scale the data they are more comparable in weight.

### Match the following situation descriptions with the model most appropriate to addressing them

In addition to logistic regression, a number of other approaches were covered this week. Pair them with the situations they are most appropriate for, and briefly explain why.

Situations:
1. You are given data on academic performance of primary school students, and asked to fit a model to help predict "at-risk" students who are likely to receive the bottom tier of grades.
2. You are studying tech companies and their patterns in releasing new products, and would like to be able to model and predict when a new product is likely to be launched.
3. You are working on modeling expected plant size and yield with a laboratory that is able to capture fantastically detailed physical data about plants, but only of a few dozen plants at a time.

Approaches:
1. Ridge Regression
2. Quantile Regression
3. Survival Analysis

**TODO - your answers!**